# Indoor Location & Navigation - Basic EDA - Paths On The Floor Map

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
from pylab import imread
from pprint import pprint

# Let's take a look into data directory.

In [ ]:
data_path = '/kaggle/input/indoor-location-navigation'
os.listdir(data_path)

* **train** directory contains sites (shopping malls) directories that consists of floor direcories. And each floor directory contains txt files, that have information about paths (smarthones).

* **metedata** directory contains floor map, its size and geo inforamation for each site and each floor.

In this competition we should predict smartphone location: floor number and x, y coordinates (TYPE_WAYPOINT). 


In [ ]:
floor = '5a0546857ecc773753327266/F1'
floor_metadata_dir = os.path.join(data_path, 'metadata', floor)
floor_train_dir = os.path.join(data_path, 'train', floor)

os.listdir(floor_metadata_dir)

In [ ]:
os.listdir(floor_train_dir)[:5]

# Loading floor map and its size

In [ ]:
image = imread(
    os.path.join(floor_metadata_dir, 'floor_image.png')
)

with open(os.path.join(floor_metadata_dir, 'floor_info.json')) as f:
    content = f.read()
    floor_info = json.loads(content)

height = float(floor_info['map_info']['height'])
width = float(floor_info['map_info']['width'])
    
pprint(floor_info)

# Loading coordinates of paths points

In [ ]:
def load_points(filepath):
    
    with open(filepath) as f:
        content = f.read()
    
    data = []
    for row in content.split('\n'):
        if 'TYPE_WAYPOINT' in row:
            data.append(row)

    points = {
        'timestamp': [],
        'x': [],
        'y': []

    }

    for row in data:
        values = row.split('\t')

        points['timestamp'].append(int(values[0]))
        points['x'].append(float(values[2]))
        points['y'].append(float(values[3]))
    
    points_df = pd.DataFrame(points)
    points_df.sort_values(by='timestamp', inplace=True)
    
    return points_df

In [ ]:
paths = []
for filename in os.listdir(floor_train_dir):
    if '.txt' not in filename:
        continue
    points = load_points(os.path.join(floor_train_dir, filename))
    paths.append((len(points), points, filename))

paths = sorted(paths, key=lambda path: path[0], reverse=True)
paths = paths[:20]

# Paths Visualization

In [ ]:
fig = plt.figure(figsize=(15, 12))
ax = plt.subplot(111)

plt.imshow(image, extent=[0, width, 0, height])



for _, points, filename in paths:
    plt.scatter(points['x'], points['y'], label=filename)
    plt.plot(points['x'], points['y'])
    

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Hope this notebook will be helpful for you. I wish you good fortune in the competition!